In [109]:
# package import
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from boruta import BorutaPy

In [101]:
#import the dataset from file path
data = pd.read_csv('Pre_Subset/transposed_GSE63063.csv',header= 0)
print(data.shape)
data.head()

(32053, 390)


,Probe_ID,Patient_barcode,GSM1539409,GSM1539410,GSM1539411,GSM1539412,GSM1539413,GSM1539414,GSM1539415,GSM1539416,...,GSM1539787,GSM1539788,GSM1539789,GSM1539790,GSM1539791,GSM1539792,GSM1539793,GSM1539794,GSM1539795,GSM1539796
0,NaN,Sex,Female,Female,Female,Female,Female,Female,Female,Female,...,Male,Male,Female,Female,Male,Female,Female,Male,Male,Female
1,NaN,Condition,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,...,AD,AD,MCI,MCI,MCI,MCI,MCI,AD,CTL,CTL
2,NaN,Ethnicity,Western European,Western European,Western European,Western European,Other Caucasian,Other Caucasian,Western European,Any_Other_White_Background,...,Western European,British_English,unkown but she's white and speaks english with...,Western European,British,Western European,Western European,Western European,Western European,British_English
3,NaN,Age,57,59,63,65,66,68,68,68,...,79,73,83,80,86,83,83,82,88,86
4,ILMN_1343291,NaN,12.55280722,12.71145907,13.08839335,12.64383051,13.09838903,13.08609968,13.40034321,12.97418372,...,12.98168176,13.63916729,12.9214465,13.2494056,13.5265911,12.31184583,13.22532792,12.95210419,13.16336147,13.34775027


In [102]:
# transpose the data such that rows and columns are flipped
data = data.T
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32043,32044,32045,32046,32047,32048,32049,32050,32051,32052
Probe_ID,NaN,NaN,NaN,NaN,ILMN_1343291,ILMN_1343295,ILMN_1651209,ILMN_1651210,ILMN_1651221,ILMN_1651228,...,ILMN_3311110,ILMN_3311115,ILMN_3311135,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311165,ILMN_3311170,ILMN_3311180,ILMN_3311190
Patient_barcode,Sex,Condition,Ethnicity,Age,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM1539409,Female,MCI,Western European,57,12.55280722,10.10155567,6.084670984,6.068804773,6.121059835,10.83398435,...,6.04762,6.19183,6.13272,6.16856,6.0649,6.09593,6.20084,6.00451,6.04553,6.25168
GSM1539410,Female,MCI,Western European,59,12.71145907,9.776015081,6.255012487,6.016468162,6.173167357,10.27067251,...,6.08059,6.20489,6.03258,6.02194,6.08126,6.08884,6.39421,5.9881,6.03702,6.16483
GSM1539411,Female,MCI,Western European,63,13.08839335,9.594397134,6.160484783,6.024321505,6.039552262,10.43059366,...,6.0661,5.98867,6.11673,6.02284,6.04491,6.14502,6.08668,6.06621,6.09827,6.28441


In [103]:
# separate the features from the labels
# patient identifiers will not be taken into account in our analyses
datax = data.drop('Patient_barcode')
# drop patient records Sex, Ethnicity, and Age
datax = datax.drop([0,2,3], axis = 1)
# Pass patient Condition into data_y to be labels
datay = (data[1][2:])
# drop the labels from datax
datax.drop(1,inplace=True,axis=1)

print(datax.shape)
print(datay.shape)
data.head()


(389, 32049)
(388,)


,0,1,2,3,4,5,6,7,8,9,...,32043,32044,32045,32046,32047,32048,32049,32050,32051,32052
Probe_ID,NaN,NaN,NaN,NaN,ILMN_1343291,ILMN_1343295,ILMN_1651209,ILMN_1651210,ILMN_1651221,ILMN_1651228,...,ILMN_3311110,ILMN_3311115,ILMN_3311135,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311165,ILMN_3311170,ILMN_3311180,ILMN_3311190
Patient_barcode,Sex,Condition,Ethnicity,Age,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM1539409,Female,MCI,Western European,57,12.55280722,10.10155567,6.084670984,6.068804773,6.121059835,10.83398435,...,6.04762,6.19183,6.13272,6.16856,6.0649,6.09593,6.20084,6.00451,6.04553,6.25168
GSM1539410,Female,MCI,Western European,59,12.71145907,9.776015081,6.255012487,6.016468162,6.173167357,10.27067251,...,6.08059,6.20489,6.03258,6.02194,6.08126,6.08884,6.39421,5.9881,6.03702,6.16483
GSM1539411,Female,MCI,Western European,63,13.08839335,9.594397134,6.160484783,6.024321505,6.039552262,10.43059366,...,6.0661,5.98867,6.11673,6.02284,6.04491,6.14502,6.08668,6.06621,6.09827,6.28441


In [104]:
# Assign probe ID to be column titles
datax.columns = (datax.iloc[0])
# drop probe ID row
datax.drop('Probe_ID',inplace=True)
datax = datax.to_numpy()


In [105]:
#print((datax.isnull().sum()).sum(),'Missing Values in the Dataset')
#substitute mean column values for missing values

# impute empty cells with the column supplied metric (string, float, or int)

def impute(dataset,metric='mean'):
    if metric == 'mean':
        dataset =dataset.fillna(dataset.mean(),inplace=False)
    elif metric == 'median':
        dataset =dataset.fillna(dataset.median(),inplace=False)
    elif metric == 'mode':
        dataset =dataset.fillna(dataset.mode(),inplace=False)
    elif metric == 'zero':
        dataset =dataset.fillna(0,inplace=False)
    elif type(metric) == float or type(metric) == int:
        dataset =dataset.fillna(metric,inplace=False)
        
    return dataset

In [106]:
datay_list = list(datay)
conds = list(set(list(datay)))
print('Counts for each class:')
for i in conds:
    print("{0}: {1}".format(i,datay_list.count(i)))

Counts for each class:
OTHER: 1
CTL to AD: 1
AD: 139
borderline MCI: 3
MCI to CTL: 1
CTL: 134
MCI: 109


In [107]:
AD = list(pd.Series.unique(datay))
CTL = AD.copy()
AD.remove("AD")
CTL.remove("CTL")
ADdatay = datay.replace(to_replace = AD, value = "nonAD")     #labels are AD and nonAD
CTLdatay = datay.replace(to_replace = CTL, value = "nonCTL")  #labels are CTL and nonCTL

In [117]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels

rfc = RandomForestClassifier(n_estimators = 10000, n_jobs = -1, class_weight = 'balanced', max_depth = 6)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, perc = 80)
boruta_selector.fit(datax, ADdatay)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	32049
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	23476
Rejected: 	8573
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	23476
Rejected: 	8573
Iteration: 	10 / 100
Confirmed: 	687
Tentative: 	22789
Rejected: 	8573
Iteration: 	11 / 100
Confirmed: 	687
Tentative: 	22789
Rejected: 	8573
Iteration: 	12 / 100
Confirmed: 	687
Tentative: 	17717
Rejected: 	13645
Iteration: 	13 / 100
Confirmed: 	916
Tentative: 	17488
Rejected: 	13645
Iteration: 	14 / 100
Confirmed: 	916
Tentative: 	17488
Rejected: 	13645
Iteration: 	15 / 100
Confirmed: 	916
Tentative: 	17

KeyboardInterrupt: 

In [ ]:
X_filtered = boruta_feature_selector.transform(X)
X_filtered.shape